In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [72]:
#this method returns:
#X_train: a list of tuples from the dataset. Each index groups X values based on kmeans++ similarity
#y_train: the corresponding target for each value in X_train

def kmeans_test_split(X,y, n_clusters):

    kmeans = KMeans(n_clusters = n_clusters, init = 'k-means++', max_iter = 300, n_init = 10)
    y_kmeans = kmeans.fit_predict(X)
    
    X_train=[]
    y_train=[]
    
    for i in range(n_clusters):
        X_train.append(X[y_kmeans == i, :])
        y_train.append(y[y_kmeans==i])
    
    return X_train,y_train
    
    #please note that I need a second look on this as it seems wrong
    #also there's some confusion about classification as kmeans gets a random order, doesn't seem relevant

In [49]:
import numpy as np
from sklearn.base import BaseEstimator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.cluster import KMeans

class CustomEstimator(BaseEstimator):
    
    def __init__(self, n_estimators, similarity_function, base_model):
        self.n_estimators = n_estimators
        self.similarity_function = similarity_function
        self.base_model = base_model
        
    
    def fit(self,X,y):
        
        #Getting a list of different test sets, temporarily using only kmeans_test_split() but of course
        #it will be based on the similarity_function value
        X_train, y_train = kmeans_test_split(X,y,self.n_estimators)
        
        #support lists to compose the estimator list that can be accepted by the voting clf
        est_clf = [] 
        est_names = []
                
        for i in range(self.n_estimators):
            
            #each test set is fitted to a different classifier
            X_selected = X_train[i]
            y_selected = y_train[i]
            
            #it is possible to expand this by creating another list of different classifiers and iterate said list
            est_clf.append(DecisionTreeClassifier(max_depth=2).fit(X_selected, y_selected))
            est_names.append("model % s" % i)
        
        #final list fed to the voting classifier
        estimators = list(zip(est_names,est_clf))
            
        
        voting_clf = VotingClassifier(estimators = estimators, voting='soft')
        self.value_ = voting_clf.fit(X,y)
        
        return self
            
    
    def predict(self, X):
        out = self.value_.predict(X)
        return out
    
    def predict_proba(self, X):
        out = self.value_.predict_proba(X)
        return out

In [70]:
#TEST ISTANCE

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
a = CustomEstimator(3, 5, 'test')
clf = a.fit(X_train,y_train)
print("prediction:", a.predict(X_test),"\n    values:",y_test)

print("prediction probabilities:\n      0     |     1     |     2\n",a.predict_proba(X_test))

from sklearn import metrics
print("Accuracy (test set):",metrics.accuracy_score(y_test, a.predict(X_test)))
print("Accuracy (full set):",metrics.accuracy_score(y, a.predict(X)))
      




prediction: [2 0 2 0 0 1 1 2 1 0 1 2 0 2 0 1 0 0 2 2 2 2 2 2 0 0 1 0 1 0] 
    values: [2 0 2 0 0 1 1 2 1 0 1 2 0 1 0 1 0 0 2 2 2 2 2 2 0 0 1 0 1 0]
prediction probabilities:
      0     |     1     |     2
 [[0.         0.         1.        ]
 [1.         0.         0.        ]
 [0.         0.         1.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         0.89361702 0.10638298]
 [0.         0.89361702 0.10638298]
 [0.         0.         1.        ]
 [0.         0.89361702 0.10638298]
 [1.         0.         0.        ]
 [0.         0.89361702 0.10638298]
 [0.         0.         1.        ]
 [1.         0.         0.        ]
 [0.         0.         1.        ]
 [1.         0.         0.        ]
 [0.         0.89361702 0.10638298]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
